<a href="https://colab.research.google.com/github/brobro10000/gpu-token-analytics-pipeline/blob/colab/CA4/producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# CA4 GPU Metadata Extraction (Google Colab)
# ==========================================

import torch
from torchvision import models, transforms
from PIL import Image
import requests, time, os, json
from pathlib import Path
import numpy as np

# -----------------------------------------------------
# 1. CONFIGURATION
# -----------------------------------------------------

# Use ngrok URL or AWS Processor API URL
EDGE_API_URL = os.getenv("EDGE_API_URL", "https://shizuko-superglottal-nonvexatiously.ngrok-free.dev")
METADATA_ENDPOINT = f"{EDGE_API_URL}/metadata"

print("Sending metadata to:", METADATA_ENDPOINT)

# -----------------------------------------------------
# 2. DEVICE SELECTION (GPU or CPU)
# -----------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------------------------------
# 3. LOAD PRETRAINED MODEL (ResNet50 feature extractor)
# -----------------------------------------------------

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = torch.nn.Identity()         # Remove classification layer
model = model.to(device)
model.eval()

# -----------------------------------------------------
# 4. IMAGE PREPROCESSOR
# -----------------------------------------------------

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

# -----------------------------------------------------
# 5. GPU METADATA EXTRACTION
# -----------------------------------------------------
# -----------------------------------------------------
# GPU HARDWARE METADATA
# -----------------------------------------------------

def get_gpu_hardware_info():
    """Returns a dictionary of GPU metadata for the CA4 payload."""
    if not torch.cuda.is_available():
        return {
            "available": False,
            "device": "cpu",
            "reason": "No CUDA device available in this runtime."
        }

    device_index = torch.cuda.current_device()
    props = torch.cuda.get_device_properties(device_index)

    return {
        "available": True,
        "device": f"cuda:{device_index}",
        "name": props.name,
        "total_memory_gb": round(props.total_memory / (1024**3), 2),
        "multi_processor_count": props.multi_processor_count,
        "cuda_compute_capability": f"{props.major}.{props.minor}",

        # current memory usage at extraction time
        "memory_allocated_mb": round(torch.cuda.memory_allocated() / (1024**2), 2),
        "memory_reserved_mb": round(torch.cuda.memory_reserved() / (1024**2), 2),

        # CUDA + PyTorch runtime metadata
        "cuda_version": torch.version.cuda,
        "pytorch_version": torch.__version__,
        "cudnn_version": torch.backends.cudnn.version(),
    }

def extract_gpu_embedding(image_path: str):
    """Returns a 2048-d embedding from ResNet50's penultimate layer."""
    img = Image.open(image_path).convert("RGB")
    x = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding_tensor = model(x)  # Shape [1, 2048]

    embedding = embedding_tensor.squeeze(0).cpu().numpy().tolist()
    return embedding


# -----------------------------------------------------
# 6. BUILD CA4 METADATA PAYLOAD
# -----------------------------------------------------

def build_payload(image_path: str, embedding: list):
    return {
        "source_type": "image",
        "source_uri": str(image_path),
        "metadata_id": f"img-{Path(image_path).stem}-{int(time.time())}",
        "model_name": "resnet50_feature_extractor",
        "model_version": "resnet50_imagenet_v1",
        "timestamp": int(time.time()),
        "embedding_dim": len(embedding),  # Should be 2048
        "embedding": embedding,
        "extra": {
            "pipeline": "ca4-gpu",
            "notes": "GPU-extracted embedding from Google Colab",
            "gpu_hardware": get_gpu_hardware_info()
        }
    }


# -----------------------------------------------------
# 7. SEND METADATA TO PROCESSOR API
# -----------------------------------------------------

def send_metadata(payload: dict):
    try:
        resp = requests.post(
            METADATA_ENDPOINT,
            json=payload,
            timeout=10
        )
        resp.raise_for_status()
        print("✓ Metadata sent successfully:", payload["metadata_id"])
        return resp.json()
    except Exception as e:
        print("❌ Error sending metadata:", e)
        return None


# -----------------------------------------------------
# 8. RUN ON A SET OF IMAGES
# -----------------------------------------------------

# Provide example images (upload or use your own)
image_paths = ["dog.png", "cat.png"]  # replace with your images

results = []

for img_path in image_paths:
    print(f"\nProcessing {img_path} ...")

    if not Path(img_path).exists():
        print(f"❌ File not found: {img_path}")
        continue

    embedding = extract_gpu_embedding(img_path)
    print(f"Extracted embedding dimension: {len(embedding)}")

    payload = build_payload(img_path, embedding)
    print(payload)
    resp = send_metadata(payload)

    results.append({"image": img_path, "response": resp})


# -----------------------------------------------------
# 9. PRINT SUMMARY
# -----------------------------------------------------

print("\n========== SUMMARY ==========\n")
for r in results:
    print(r)


Sending metadata to: https://shizuko-superglottal-nonvexatiously.ngrok-free.dev/metadata
Using device: cuda

Processing dog.png ...
Extracted embedding dimension: 2048
{'source_type': 'image', 'source_uri': 'dog.png', 'metadata_id': 'img-dog-1764873316', 'model_name': 'resnet50_feature_extractor', 'model_version': 'resnet50_imagenet_v1', 'timestamp': 1764873316, 'embedding_dim': 2048, 'embedding': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2008601576089859, 0.004519355483353138, 0.0, 0.3504452109336853, 0.0, 0.0, 0.0027775957714766264, 0.0, 0.0, 0.0, 0.008800248615443707, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.026039116084575653, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13664518296718597, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08576281368732452, 0.0, 0.0, 0.0, 0.0, 0.0764496698975563, 0.0, 0.0, 0.0, 0.02931421622633934, 0.0, 0.15012595057487488, 0.0, 0.012928295880556107, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000836929539218545, 0